# Redes

Muchos temas apasionantes, como la visión por ordenador, el Internet de las cosas (IoT) y los bots de chat con IA, dominan los titulares. Esto hace que se olviden las tecnologías básicas tradicionales. Aunque es fantástico tener muchas capacidades nuevas al alcance de la mano, es importante darse cuenta de que estas tecnologías no serían posibles sin una base sólida de conectividad fiable y segura. El procesamiento de datos sólo es útil si los resultados se entregan de forma fiable y son accesibles a través de una red. Los contenedores son un método fantástico de despliegue de aplicaciones, pero proporcionan la mejor experiencia para los usuarios cuando están conectados en red.

Los servicios y características de red dentro de AWS son la columna vertebral de casi todos los grandes servicios que cubrimos en este libro. AWS tiene muchas características excelentes para que conectes lo que quieras, donde quieras y como quieras. Obtener una mejor comprensión de las redes le permitirá tener una mejor comprensión de la nube y por lo tanto estar más cómodo usándola.

In [1]:
import boto3
import json

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

### Definición de su red virtual privada en la nube mediante la creación de una VPC de Amazon

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0201.png" width="650">

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.16.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBook"}])
vpc.wait_until_available()

*Cuando se crea una VPC, la documentación indica que el mayor tamaño de bloque para los CIDR IPv4 de la VPC es una máscara de red /16 (65.536 direcciones IP). El más pequeño es una máscara de red /28 (16 direcciones IP).*

In [3]:
vpc_decribe = ec2_client.describe_vpcs(VpcIds=[vpc.id])["Vpcs"][0]
print(json.dumps(vpc_decribe, sort_keys=True, indent=4))

{
    "CidrBlock": "174.16.0.0/16",
    "CidrBlockAssociationSet": [
        {
            "AssociationId": "vpc-cidr-assoc-04a431a9b0b899e81",
            "CidrBlock": "174.16.0.0/16",
            "CidrBlockState": {
                "State": "associated"
            }
        }
    ],
    "DhcpOptionsId": "dopt-023e90ceacfaf337c",
    "InstanceTenancy": "default",
    "IsDefault": false,
    "OwnerId": "899696702655",
    "State": "available",
    "Tags": [
        {
            "Key": "Name",
            "Value": "AWSCookBook"
        }
    ],
    "VpcId": "vpc-0c4672fb3b1b5a4be"
}


Hay dos razones importantes para seleccionar cuidadosamente los bloques CIDR para su VPC:

* Una vez que un bloque CIDR está asociado a una VPC, no puede ser modificado (aunque puede ser ampliado). Si desea modificar un bloque CIDR, deberá eliminarlo (y todos los recursos que contiene) y volver a crearlo.

* Si una VPC está conectada a otras redes mediante peering o pasarelas (por ejemplo, Tránsito y VPN), tener rangos de IP superpuestos causará problemas no deseados.

Puede añadir espacio IPv4 a la VPC utilizando el comando aws ec2 associate-vpc-cidr-block para especificar el espacio IPv4 adicional. Cuando el espacio IP escasea por el uso y el subaprovisionamiento, es bueno saber que no es necesario dedicar un gran bloque a una VPC, especialmente si no está seguro de que se utilizará todo.

In [4]:
# Este es un ejemplo de asociación de un bloque CIDR IPv4 adicional a su VPC:
ec2_client.associate_vpc_cidr_block(
    VpcId=vpc.id,
    CidrBlock='174.17.0.0/16'
)

{'CidrBlockAssociation': {'AssociationId': 'vpc-cidr-assoc-058f6ede9746e6a5c',
  'CidrBlock': '174.17.0.0/16',
  'CidrBlockState': {'State': 'associating'}},
 'VpcId': 'vpc-0c4672fb3b1b5a4be',
 'ResponseMetadata': {'RequestId': 'c2677a48-1ccd-4823-8c26-1de18d0580e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2677a48-1ccd-4823-8c26-1de18d0580e3',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Sat, 01 Oct 2022 17:06:31 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

Amazon VPC admite direcciones IPv4 e IPv6 y tiene cuotas de tamaño de bloque de CIDR diferentes para cada una de ellas. De forma predeterminada, todas las VPC y subredes deben tener bloques de CIDR IPv4. Esto no se puede cambiar. Si lo desea, puede asociar un bloque de CIDR IPv6 a su VPC especificando la opción --amazon-provided-ipv6-cidr-block. 

Este es un ejemplo de creación de una VPC con un bloque CIDR IPv6:

In [5]:
vpc = ec2.create_vpc(CidrBlock='10.10.0.0/16', AmazonProvidedIpv6CidrBlock=True)
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBook-IPv6"}])
vpc.wait_until_available()

Una VPC es una construcción regional en AWS. Una región es un área geográfica, y las zonas de disponibilidad son centros de datos físicos que residen en una región. Las regiones abarcan todas las zonas de disponibilidad (AZ), que son grupos de centros de datos físicos aislados. El número de AZs por región varía, pero todas las regiones tienen al menos tres. Para obtener la información más actualizada sobre las regiones y las zonas de disponibilidad de AWS, consulte este artículo sobre ["Regiones y zonas de disponibilidad"](https://aws.amazon.com/es/about-aws/global-infrastructure/regions_az/).

Según la guía del usuario de la VPC, la cuota inicial de bloques CIDR IPv4 por VPC es de 5. Puede aumentarse a 50. El número permitido de bloques CIDR IPv6 por VPC es 1.

### Creación de un nivel de red con subredes y una tabla de rutas en una VPC

Cree una tabla de rutas dentro de su VPC. Cree dos subredes en zonas de disponibilidad separadas en una VPC. Asocie la tabla de rutas con las subredes.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0202.png" width="650">


In [6]:
# Crear una tabla de rutas. Esto le permitirá crear rutas de tráfico personalizadas para las subredes asociadas:
routetable = vpc.create_route_table()
routetable.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT"}])

[ec2.Tag(resource_id='rtb-0842226dcf949f4fd', key='Name', value='AWSCookbook-RT')]

In [7]:
# Cree dos subredes, una en cada AZ. Esto definirá los espacios de direcciones para que pueda crear recursos para su VPC:
subnet_1 = ec2.create_subnet(
    CidrBlock='10.10.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet_2 = ec2.create_subnet(
    CidrBlock='10.10.1.0/24',
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)

In [8]:
subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Subnet-1"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Subnet-2"}])

[ec2.Tag(resource_id='subnet-0fb0893334dbb2abf', key='Name', value='AWSCookbook-Subnet-2')]

En los comandos anteriores, el parámetro `--availability-zone` utiliza una variable de entorno para su región añadiendo caracteres a o b en minúsculas para indicar qué AZ lógica (por ejemplo, us-east-1a) debe aprovisionar cada subred. AWS afirma que estos nombres son aleatorios por cuenta para equilibrar los recursos entre las AZ.

Si utiliza varias cuentas de AWS y desea encontrar ID de zona de disponibilidad para una región que sean coherentes, ejecute este comando:

In [9]:
for i in ec2_client.describe_availability_zones()["AvailabilityZones"]:
    print(i["ZoneName"])

us-east-1a
us-east-1b
us-east-1c
us-east-1d
us-east-1e
us-east-1f


In [10]:
# Asociar la tabla de rutas con las dos subredes:
routetable.associate_with_subnet(SubnetId=subnet_1.id)
routetable.associate_with_subnet(SubnetId=subnet_2.id)

ec2.RouteTableAssociation(id='rtbassoc-013a304c524ace097')

In [11]:
# Recupere la configuración de las subredes que creó y verifique que están en la misma VPC pero en diferentes AZs:
subnet_data = ec2_client.describe_subnets(SubnetIds=[subnet_1.id, subnet_2.id])['Subnets']
print(json.dumps(subnet_data, sort_keys=True, indent=4))

[
    {
        "AssignIpv6AddressOnCreation": false,
        "AvailabilityZone": "us-east-1b",
        "AvailabilityZoneId": "use1-az2",
        "AvailableIpAddressCount": 251,
        "CidrBlock": "10.10.1.0/24",
        "DefaultForAz": false,
        "EnableDns64": false,
        "Ipv6CidrBlockAssociationSet": [],
        "Ipv6Native": false,
        "MapCustomerOwnedIpOnLaunch": false,
        "MapPublicIpOnLaunch": false,
        "OwnerId": "899696702655",
        "PrivateDnsNameOptionsOnLaunch": {
            "EnableResourceNameDnsAAAARecord": false,
            "EnableResourceNameDnsARecord": false,
            "HostnameType": "ip-name"
        },
        "State": "available",
        "SubnetArn": "arn:aws:ec2:us-east-1:899696702655:subnet/subnet-0fb0893334dbb2abf",
        "SubnetId": "subnet-0fb0893334dbb2abf",
        "Tags": [
            {
                "Key": "Name",
                "Value": "AWSCookbook-Subnet-2"
            }
        ],
        "VpcId": "vpc-09ebcbcb23

In [12]:
# Valida que la tabla de rutas que has creado está asociada a las dos subredes:
ec2_client.describe_route_tables(RouteTableIds=[routetable.id])['RouteTables']

[{'Associations': [{'Main': False,
    'RouteTableAssociationId': 'rtbassoc-0c353921fcee53082',
    'RouteTableId': 'rtb-0842226dcf949f4fd',
    'SubnetId': 'subnet-08dbe94610dcd0477',
    'AssociationState': {'State': 'associated'}},
   {'Main': False,
    'RouteTableAssociationId': 'rtbassoc-013a304c524ace097',
    'RouteTableId': 'rtb-0842226dcf949f4fd',
    'SubnetId': 'subnet-0fb0893334dbb2abf',
    'AssociationState': {'State': 'associated'}}],
  'PropagatingVgws': [],
  'RouteTableId': 'rtb-0842226dcf949f4fd',
  'Routes': [{'DestinationCidrBlock': '10.10.0.0/16',
    'GatewayId': 'local',
    'Origin': 'CreateRouteTable',
    'State': 'active'},
   {'DestinationIpv6CidrBlock': '2600:1f18:7d8:cb00::/56',
    'GatewayId': 'local',
    'Origin': 'CreateRouteTable',
    'State': 'active'}],
  'Tags': [{'Key': 'Name', 'Value': 'AWSCookbook-RT'}],
  'VpcId': 'vpc-09ebcbcb231eb359a',
  'OwnerId': '899696702655'}]

Al diseñar una estrategia de subred, debe elegir tamaños de subred que se ajusten a sus necesidades actuales y tengan en cuenta el crecimiento futuro de su aplicación. Las subredes se utilizan para la colocación de la interfaz de red elástica (ENI) para los recursos de AWS. Esto significa que una ENI concreta vive dentro de una única AZ.

AWS reserva las cuatro primeras y la última dirección IP del bloque CIDR de cada subred para características y funcionalidades cuando se crea una subred. Estas no están disponibles para su uso. Según la documentación, estas son las direcciones reservadas en el caso de su ejemplo:

* *.0* - Dirección de red.
* *.1* - Reservada por AWS para el router de la VPC.
* *.2* - Reservada por AWS para la dirección IP del servidor DNS. Siempre se establece el rango de red de la VPC más dos.
* *.3* - Reservada por AWS para uso futuro.
* *.255* - Dirección de difusión de la red. No se admite la difusión en una VPC.

Una subred tiene una tabla de rutas asociada a ella. Las tablas de rutas pueden estar asociadas a una o más subredes y dirigir el tráfico a un destino de su elección (más adelante se hablará de esto con las recetas de pasarela NAT, gateway de Internet y gateway de tránsito). Las entradas dentro de las tablas de rutas se llaman rutas y se definen como pares de Destinos y Objetivos. Al crear la tabla de rutas, se añadió automáticamente una ruta local por defecto que gestiona el tráfico intra-VPC. Tiene la posibilidad de crear rutas personalizadas que se ajusten a sus necesidades. Para obtener una lista completa de los destinos disponibles para utilizar dentro de las tablas de rutas, consulte este [documento de soporte](https://oreil.ly/oKVq1).

*Las ENIs reciben una dirección IP de un servidor DHCP administrado por AWS dentro de su VPC. El conjunto de opciones DHCP se configura automáticamente con valores predeterminados para asignar direcciones dentro de las subredes que usted defina.*


Cuando se crea una VPC en una región, es una buena práctica repartir las subredes entre las AZ de ese nivel de red. El número de AZs varía según la región, pero la mayoría tienen al menos tres. Un ejemplo de esto en la práctica sería que si tuvieras un tier público y un tier aislado repartidos en dos AZs, tendrías un total de cuatro subredes: 2 tiers × 2 subredes por tier (una por AZ).

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0203.png" width="600">

### Conexión de su VPC a Internet mediante un gateway de Internet

Crearás un gateway de internet y lo adjuntarás a tu VPC. A continuación, modificará la tabla de rutas asociada a la subred en la que se encuentra la instancia EC2. Añadirás una ruta que envíe el tráfico de las subredes a la gateway de Internet. Por último, crea una IP elástica (EIP) y asóciala a la instancia:

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0204.png" width="700">

In [13]:
# Crear un gateway de Internet (IGW):
internetgateway = ec2.create_internet_gateway(
    TagSpecifications=[{"ResourceType": "internet-gateway", 
    "Tags": [{
            "Key": "Name", 
            "Value": "AWSCookbook-IGW"}
            ]}
    ]
)

In [14]:
# Adjunte el gateway de Internet a la VPC existente:
vpc.attach_internet_gateway(InternetGatewayId=internetgateway.id)

{'ResponseMetadata': {'RequestId': '7e546f61-9cca-4223-a0f7-d29e5a4444d4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e546f61-9cca-4223-a0f7-d29e5a4444d4',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Sat, 01 Oct 2022 17:07:10 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [15]:
# En la tabla de rutas de su VPC, cree una ruta que establezca el destino de la ruta por defecto a la gateway de Internet:
route = routetable.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=internetgateway.id)

In [16]:
# Crear una EIP (dirección IP elástica):
allocation_id = ec2_client.allocate_address(Domain='vpc')['AllocationId']

*AWS define una dirección IP elástica (EIP) como "una dirección IPv4 estática diseñada para la computación dinámica en la nube. Una dirección EIP se asigna a su cuenta de AWS y es suya hasta que la libere".*

In [ ]:
# Asocie el EIP con la instancia EC2 existente:
ec2_client.associate_address(
    AllocationId = allocation_id, 
    InstanceId = 'i-00b555777daa2e67c'
)

In [ ]:
# Haz ping a un host en Internet para probar la conectividad a Internet:
ping -c 4 homestarrunner.com

La ruta que ha creado en su entrada de la tabla de rutas envía todo el tráfico no local al IGW que proporciona la conectividad a Internet de su VPC. Debido a que estaba trabajando con una instancia EC2 existente en ejecución, necesitaba crear una IP elástica y asociarla con la instancia. Estos pasos permitieron la comunicación por Internet con la instancia sin tener que interactuar con ella. Existe una opción para habilitar la autoasignación de direcciones IPv4 públicas para las instancias recién lanzadas en una subred. Sin embargo, si utiliza la autoasignación, las IPs públicas cambiarán después de cada reinicio de la instancia. Las EIPs asociadas a una instancia no cambiarán tras los reinicios.

*NOTA: Las tablas de rutas dan prioridad a la ruta más específica. AWS también permite crear rutas más específicas que la ruta local por defecto.*

El grupo de seguridad asociado a su instancia no permite el acceso entrante. Si desea permitir el acceso entrante a Internet a una instancia en una subred pública, tendrá que configurar una regla de entrada de grupo de seguridad para ello.

Una subred que tenga una ruta 0.0.0.0/0 asociada a un IGW se considera una subred pública. Se considera una práctica recomendada de seguridad colocar sólo en este tipo de nivel instancias que requieran acceso entrante desde la Internet pública. Los equilibradores de carga orientados al usuario final suelen colocarse en subredes públicas. Una subred pública no sería una opción ideal para un servidor de aplicaciones o una base de datos. En estos casos, puede crear un nivel privado o un nivel aislado que se adapte a sus necesidades con el enrutamiento adecuado y utilizar una gateway NAT para dirigir el tráfico de la subred a la puerta de enlace de Internet sólo cuando se requiera el acceso a Internet saliente.

### Uso de una gateway NAT para el acceso saliente a Internet desde subredes privadas

*Problema*
Usted tiene subredes públicas en su VPC que tienen una ruta a un gateway de Internet. Desea aprovechar esta configuración para proporcionar acceso a Internet sólo de salida para una instancia en subredes privadas.

*Solución*
Cree un gateway NAT en una de las subredes públicas. A continuación, cree una IP elástica y asóciela a la gateway NAT. En la tabla de rutas asociada a las subredes privadas, añada una ruta para el tráfico de salida a Internet que tenga como objetivo la gateway NAT.

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0205.png" width="700">

In [ ]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.16.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBook"}])
vpc.wait_until_available()

In [3]:
# Crear routables
routetable_public = vpc.create_route_table()
routetable_public.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Public"}])

routetable_private1 = vpc.create_route_table()
routetable_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Private1"}])

routetable_private2 = vpc.create_route_table()
routetable_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-RT-Private2"}])

[ec2.Tag(resource_id='rtb-073751320771e8ce3', key='Name', value='AWSCookbook-RT-Private2')]

In [5]:
# Cree cuatro subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.16.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet_2 = ec2.create_subnet(
    CidrBlock='174.16.1.0/24',
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)
subnet_3 = ec2.create_subnet(
    CidrBlock='174.16.2.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet_4 = ec2.create_subnet(
    CidrBlock='174.16.3.0/24',
    VpcId=vpc.id,
    AvailabilityZone='us-east-1b'
)

In [6]:
subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Public-SN"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Public-SN"}])
subnet_3.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Private-SN"}])
subnet_4.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Private-SN"}])

[ec2.Tag(resource_id='subnet-0c28e67b99af7fc65', key='Name', value='AWSCookbook-Private-SN')]

In [7]:
# Crear un gateway de Internet (IGW):
internetgateway = ec2.create_internet_gateway(
    TagSpecifications=[{"ResourceType": "internet-gateway", 
    "Tags": [{
            "Key": "Name", 
            "Value": "AWSCookbook-IGW"}
            ]}
    ]
)

In [8]:
# Adjunte el gateway de Internet a la VPC existente:
vpc.attach_internet_gateway(InternetGatewayId=internetgateway.id)

{'ResponseMetadata': {'RequestId': 'ea4b1fe8-dd41-4461-afdb-d3aa7cca47e9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea4b1fe8-dd41-4461-afdb-d3aa7cca47e9',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Mon, 03 Oct 2022 16:52:09 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}